In [2]:
%load_ext autoreload 
%autoreload 2

from preprocessing import FileIO, Vectorizor
from opensearch_interface import OpenSearchClient
from index_templates import youtube_body

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
model_path = 'thenlper/gte-base' 
osclient = OpenSearchClient(model_name_or_path=model_path)

In [7]:
osclient.info()

{'name': 'aa5a177f3093',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '3B8leOuZTg6pKrDUDoaFUQ',
 'version': {'distribution': 'opensearch',
  'number': '2.10.0',
  'build_type': 'tar',
  'build_hash': 'eee49cb340edc6c4d489bcd9324dda571fc8dc03',
  'build_date': '2023-09-20T23:54:29.889267151Z',
  'build_snapshot': False,
  'lucene_version': '9.7.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [8]:
osclient.show_indexes()

health status index                     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .plugins-ml-config        RN1M3CHMQdaLyBA4SyOKrQ   1   0          1            0      3.9kb          3.9kb
green  open   .opensearch-observability g7BD1x-TSR27YeCaHl1haQ   1   0          0            0       208b           208b
green  open   .opendistro_security      qmiUP8fXRcGM8Q-9b0seEw   1   0         10            0     76.2kb         76.2kb



In [9]:
data_path = './data/impact_theory_gte_196.parquet'

In [11]:
data = FileIO().load_parquet(data_path)

Shape of data: (9055, 16)
Memory Usage: 1.05+ MB


In [12]:
sorted(list(data[0].keys()))

['age_restricted',
 'author',
 'channel_id',
 'content',
 'content_embedding',
 'description',
 'doc_id',
 'episode_num',
 'keywords',
 'length',
 'playlist_id',
 'publish_date',
 'thumbnail_url',
 'title',
 'video_id',
 'views']

### Data Indexing

In [15]:
index_name = 'impact-theory-gte-196'

In [16]:
youtube_body

{'settings': {'number_of_shards': 3,
  'refresh_interval': '30s',
  'index': {'knn': False}},
 'mappings': {'properties': {'title': {'type': 'text', 'index': 'true'},
   'unique_id': {'type': 'keyword', 'index': 'false'},
   'group_id': {'type': 'short', 'index': 'false'},
   'video_id': {'type': 'keyword', 'index': 'false'},
   'playlist_id': {'type': 'keyword', 'index': 'false'},
   'episode_url': {'type': 'keyword', 'index': 'false'},
   'episode_num': {'type': 'short', 'index': 'false'},
   'description': {'type': 'text', 'index': 'true'},
   'length': {'type': 'long', 'index': 'false'},
   'publish_date': {'type': 'keyword', 'index': 'false'},
   'views': {'type': 'long', 'index': 'false'},
   'thumbnail_url': {'type': 'keyword', 'index': 'false'},
   'content': {'type': 'text', 'index': 'true'},
   'doc_id': {'type': 'keyword', 'index': 'false'}}}}

In [17]:
%%time
osclient.document_indexer(index_name=index_name, data=data, chunk_size=1500, body_template=youtube_body, semantic_index=True)

2023-10-14 20:43:13.786 | INFO     | opensearch_interface:document_indexer:212 - The ** impact-theory-gte-196 ** index was created
2023-10-14 20:43:13.787 | INFO     | opensearch_interface:document_indexer:219 - The # of documents to be indexed = 9055
100%|██████████| 9055/9055 [00:17<00:00, 513.91Docs Indexed/s]

CPU times: user 6.14 s, sys: 633 ms, total: 6.77 s
Wall time: 18.6 s


## Test Search on Index

In [18]:
query = 'How do I change my life for the better'

In [19]:
kw_response = osclient.keyword_search(query=query, index=index_name, size=5)
kw_response

[{'_index': 'impact-theory-gte-196',
  '_id': 'hVHtL4sBeAHUcbGk8TcU',
  '_score': 14.285405,
  '_source': {'length': 7266,
   'episode_num': 351,
   'title': "Alzheimer's Disease: The EARLY WARNING SIGNS & How To Reverse It | Richard Johnson & Dale Bredesen",
   'thumbnail_url': 'https://i.ytimg.com/vi/Zn4MWA1BESY/hq720.jpg',
   'doc_id': 'Zn4MWA1BESY_41',
   'publish_date': '05-04-2023',
   'content': "It makes all the difference So for brain health You want a plant-rich? Ketogenic diet with appropriate periods of fasting so you want to get the ketones up you want to have the low-carb diet But you don't want to have just a pure meat diet. Why is that? So I'm ultra high I get most of my calories from meat. So if you're gonna So change my life right now. I want to hear about it. So what what is the difference? How how does it work mechanistically? Let's change your future for the better. Absolutely.",
   'views': 477365,
   'video_id': 'Zn4MWA1BESY'}},
 {'_index': 'impact-theory-gte-196

In [20]:
vec_response = osclient.vector_search(query, index_name, size=5)
vec_response

[{'_index': 'impact-theory-gte-196',
  '_id': 'olHtL4sBeAHUcbGk3ixQ',
  '_score': 0.7880753,
  '_source': {'length': 12406,
   'episode_num': 372,
   'title': 'Turning Boys Into Men: How To Stop Being WEAK & Become A 1% Man  | Tom Bilyeu',
   'thumbnail_url': 'https://i.ytimg.com/vi/WhLdpjZjUrw/hq720.jpg',
   'doc_id': 'WhLdpjZjUrw_86',
   'publish_date': '06-29-2023',
   'content': "Change your habits and define what you mean by better person. So this goes back to the idea that the most important thing anyone can have in their life is clarity. So who is it that you're actually trying to become? The important thing is to make sure that you know what your value system is, you know what you value, then we know what you're actually striving towards. Once you have a goal, that goal makes demands. So if you're trying to be the kind of person that gets up early so that you can see your kids before you send them off to school, then you need to be the kind of person that either deals with fati